In [3]:
#import bk data
try:
    from xml.etree import cElementTree as ET
except ImportError, e:
    from xml.etree import ElementTree as ET


def extract_intersections(osm, verbose=True):
    # This function takes an osm file as an input. It then goes through each xml 
    # element and searches for nodes that are shared by two or more ways.
    # Parameter:
    # - osm: An xml file that contains OpenStreetMap's map information
    # - verbose: If true, print some outputs to terminal.
    # 
    # Ex) extract_intersections('WashingtonDC.osm')
    #
    tree = ET.parse(osm)
    root = tree.getroot()
    counter = {}
    way_counter = {}
    node_intersections = {}
    for child in root:
        if child.tag == 'way':
            hwstat = False
            way_node = []
            #while hwstat == False:
            for item in child:
                if item.tag == 'tag' and item.attrib['k'] == 'highway':
                    way_id = child.attrib['id']
               #     way_name = item.attrib['v'] pull some name data to save later!
                    hwstat = True
            if hwstat == True:
                for item in child:
                    if item.tag == 'nd':
                        nd_ref = item.attrib['ref']
                        way_node.append(nd_ref)
                        if not nd_ref in counter:
                            counter[nd_ref] = 0
                        counter[nd_ref] += 1
                        #way_counter[way_id] = []
                way_counter[way_id] = way_node
            else:
                hwstat = False
    # Find nodes that are shared with more than one way, which
    # might correspond to intersections
    intersections = filter(lambda x: counter[x] > 1,  counter)

    # Extract intersection coordinates
    # You can plot the result using this url.
    # http://www.darrinward.com/lat-long/
    intersection_coordinates = []
    for child in root:
        if child.tag == 'node' and child.attrib['id'] in intersections:
            coordinate = child.attrib['lat'] + ',' + child.attrib['lon']
            if verbose:
                print coordinate
            intersection_coordinates.append(coordinate)
            node_intersections[child.attrib['id']] = (child.attrib['lat'], child.attrib['lon'])

    return intersection_coordinates, way_counter, node_intersections

In [4]:
filename = 'map_extrasmall.osm'
int_string, ways, nodes = extract_intersections(filename, verbose= False)


In [1]:
import networkx as nx

In [2]:
G = nx.Graph()

In [5]:
for d in nodes:
    G.add_node(d)

In [9]:
#G.nodes()
G.number_of_nodes()

217

In [10]:
ways2 = {}
for w in ways:
    num_el = len(ways[w])
    if num_el == 2:
        ways2[w]=ways[w]
    elif num_el > 2:
        for j in range(0,num_el-1):
            
            new_list= [ways[w][j], ways[w][j+1]]
            ways2[w+str(j)] = new_list
            

In [17]:

for w in ways2:
    G.add_edge(ways2[w][0],ways2[w][1])

In [19]:
import matplotlib.pylab as plt


In [22]:
nx.draw(G)
plt.show()

In [50]:
from geopy.distance import great_circle
from geopy.distance import vincenty

In [51]:
node1 = nodes['42461596']
node2 = nodes['42461598']
#node2 = nodes[ways2['1073911040'][1]]
#print node1, node2
print great_circle(node1, node2).miles
print vincenty(node1,node2).miles

0.0216576132347
0.021706555656


In [54]:
#determining weights
for w in ways2:
    print great_circle(nodes[ways2[w][0]],nodes[ways2[w][1]]).miles

0.0734910113786


KeyError: '275057630'